In [1]:
import sys
import talib as talib
import pandas as pd
import numpy as np
from pyecharts import options as opts
sys.path.append("..") 
from imp import reload
from src.ts_stock import TsStock
from src.chart import Chart
import tushare as ts
tsStock = TsStock();
chart = Chart();
token='36d50a0386e9066b5c79c2de58ea2d3a9b980020041adc910f76bfc7'
ts.set_token(token)
pro=ts.pro_api()


/Users/david/anaconda3/lib/python3.6/site-packages/tushare/util/upass.py:28: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  return str(df.ix[0]['token'])


In [2]:
stock = '603816'
start_date = '20080101'
end_date=''

In [3]:
stock_info = tsStock.get_stock_info(stock)
df_income = pro.income(ts_code=stock_info['code'], start_date=start_date, end_date=end_date)
#df_income.to_excel("income.xlsx",sheet_name='Sheet_name_1')
df_income['n_income_attr_p']= df_income.loc[:,'n_income_attr_p']/100000000.0
df_income['operate_profit'] = df_income['operate_profit']/100000000.0
df_income.index = pd.to_datetime( df_income.ann_date )
df_income = df_income.sort_index(ascending=True)

In [4]:
c = chart.get_line_income(df_income,stock_info)
c.render_notebook()

In [5]:
df_income


,ts_code,ann_date,f_ann_date,end_date,report_type,comp_type,basic_eps,diluted_eps,total_revenue,revenue,...,minority_gain,oth_compr_income,t_compr_income,compr_inc_attr_p,compr_inc_attr_m_s,ebit,ebitda,insurance_exp,undist_profit,distable_profit
ann_date,,,,,,,,,,,,,,,,,,,,,
2014-05-05,603816.SH,20140505,20140505,20111231,1,1,1.14,1.14,2.062643e+09,2.062643e+09,...,NaN,NaN,NaN,NaN,NaN,4.139620e+08,4.139620e+08,None,None,None
2015-06-26,603816.SH,20150626,20150626,20121231,1,1,0.90,0.90,2.118024e+09,2.118024e+09,...,NaN,4.630000e+05,2.974841e+08,2.974841e+08,NaN,3.129338e+08,3.129338e+08,None,None,None
2016-09-19,603816.SH,20160919,20160919,20131231,1,1,1.22,1.22,2.681748e+09,2.681748e+09,...,-2973428.24,-1.184177e+06,3.994729e+08,4.024464e+08,-2973428.24,4.588570e+08,4.973172e+08,None,None,None
2016-09-19,603816.SH,20160919,20160919,20141231,1,1,1.29,1.29,3.239714e+09,3.239714e+09,...,-4152817.16,-1.923519e+05,4.200253e+08,4.241781e+08,-4152817.16,5.331062e+08,5.807383e+08,None,None,None
2016-09-19,603816.SH,20160919,20160919,20151231,1,1,1.51,1.51,3.684846e+09,3.684846e+09,...,-7785046.57,-1.577072e+05,4.903691e+08,4.981542e+08,-7785046.57,5.761595e+08,6.287400e+08,None,None,None
2016-09-19,603816.SH,20160919,20160919,20160630,1,1,0.82,0.82,2.011948e+09,2.011948e+09,...,2729974.16,-2.240948e+05,2.745656e+08,2.718170e+08,2748625.51,3.410446e+08,3.787060e+08,None,None,None
2016-10-29,603816.SH,20161029,20161029,20150930,1,1,1.04,1.04,2.567256e+09,2.567256e+09,...,-5055468.46,-7.012978e+04,3.393355e+08,3.443910e+08,-5055468.46,NaN,NaN,None,None,None
2016-10-29,603816.SH,20161029,20161029,20160930,1,1,1.27,1.27,3.248275e+09,3.248275e+09,...,4793802.79,1.028830e+07,4.347136e+08,4.299011e+08,4812454.14,NaN,NaN,None,None,None
2017-03-21,603816.SH,20170321,20170321,20161231,1,1,1.67,1.67,4.794535e+09,4.794535e+09,...,-2596537.38,9.274364e+06,5.817294e+08,5.842711e+08,-2541692.16,6.512256e+08,7.317624e+08,None,None,None


In [6]:
df_income

,ts_code,ann_date,f_ann_date,end_date,report_type,comp_type,basic_eps,diluted_eps,total_revenue,revenue,...,minority_gain,oth_compr_income,t_compr_income,compr_inc_attr_p,compr_inc_attr_m_s,ebit,ebitda,insurance_exp,undist_profit,distable_profit
ann_date,,,,,,,,,,,,,,,,,,,,,
2014-05-05,603816.SH,20140505,20140505,20111231,1,1,1.14,1.14,2.062643e+09,2.062643e+09,...,NaN,NaN,NaN,NaN,NaN,4.139620e+08,4.139620e+08,None,None,None
2015-06-26,603816.SH,20150626,20150626,20121231,1,1,0.90,0.90,2.118024e+09,2.118024e+09,...,NaN,4.630000e+05,2.974841e+08,2.974841e+08,NaN,3.129338e+08,3.129338e+08,None,None,None
2016-09-19,603816.SH,20160919,20160919,20131231,1,1,1.22,1.22,2.681748e+09,2.681748e+09,...,-2973428.24,-1.184177e+06,3.994729e+08,4.024464e+08,-2973428.24,4.588570e+08,4.973172e+08,None,None,None
2016-09-19,603816.SH,20160919,20160919,20141231,1,1,1.29,1.29,3.239714e+09,3.239714e+09,...,-4152817.16,-1.923519e+05,4.200253e+08,4.241781e+08,-4152817.16,5.331062e+08,5.807383e+08,None,None,None
2016-09-19,603816.SH,20160919,20160919,20151231,1,1,1.51,1.51,3.684846e+09,3.684846e+09,...,-7785046.57,-1.577072e+05,4.903691e+08,4.981542e+08,-7785046.57,5.761595e+08,6.287400e+08,None,None,None
2016-09-19,603816.SH,20160919,20160919,20160630,1,1,0.82,0.82,2.011948e+09,2.011948e+09,...,2729974.16,-2.240948e+05,2.745656e+08,2.718170e+08,2748625.51,3.410446e+08,3.787060e+08,None,None,None
2016-10-29,603816.SH,20161029,20161029,20150930,1,1,1.04,1.04,2.567256e+09,2.567256e+09,...,-5055468.46,-7.012978e+04,3.393355e+08,3.443910e+08,-5055468.46,NaN,NaN,None,None,None
2016-10-29,603816.SH,20161029,20161029,20160930,1,1,1.27,1.27,3.248275e+09,3.248275e+09,...,4793802.79,1.028830e+07,4.347136e+08,4.299011e+08,4812454.14,NaN,NaN,None,None,None
2017-03-21,603816.SH,20170321,20170321,20161231,1,1,1.67,1.67,4.794535e+09,4.794535e+09,...,-2596537.38,9.274364e+06,5.817294e+08,5.842711e+08,-2541692.16,6.512256e+08,7.317624e+08,None,None,None


In [7]:
df = df_income.drop_duplicates(["ann_date"],keep="last")[['n_income_attr_p']]
df

,n_income_attr_p
ann_date,
2014-05-05,3.755677
2015-06-26,2.970211
2016-09-19,2.720597
2016-10-29,4.196315
2017-03-21,5.750515
2017-04-28,1.878487
2017-08-23,3.879871
2017-10-31,6.200081
2018-04-26,8.224456


In [8]:
df['last_year_n_income_attr_p'] = df['n_income_attr_p'].shift(4)
df['last_n_income_attr_p'] = df['n_income_attr_p'].shift(1)

df

,n_income_attr_p,last_year_n_income_attr_p,last_n_income_attr_p
ann_date,,,
2014-05-05,3.755677,NaN,NaN
2015-06-26,2.970211,NaN,3.755677
2016-09-19,2.720597,NaN,2.970211
2016-10-29,4.196315,NaN,2.720597
2017-03-21,5.750515,3.755677,4.196315
2017-04-28,1.878487,2.970211,5.750515
2017-08-23,3.879871,2.720597,1.878487
2017-10-31,6.200081,4.196315,3.879871
2018-04-26,8.224456,5.750515,6.200081


In [9]:
df['income_tongbi'] =  ( df['n_income_attr_p']- df['last_year_n_income_attr_p'] )/(df['last_year_n_income_attr_p'])

In [10]:
df['income_huanbi'] = (df['n_income_attr_p']- df['last_n_income_attr_p'] )/ df['last_n_income_attr_p']

In [11]:
df.rename(columns={'n_income_attr_p': 'n_income'}, inplace=True)

In [12]:
df.tail(1)

,n_income,last_year_n_income_attr_p,last_n_income_attr_p,income_tongbi,income_huanbi
ann_date,,,,,
2019-08-15,5.587085,4.825069,2.95385,0.157929,0.891459


In [13]:
df = pro.concept()

In [14]:
df

,code,name,src
0,TS0,密集调研,ts
1,TS1,南北船合并,ts
2,TS2,5G,ts
3,TS3,机场,ts
4,TS4,高价股,ts
...,...,...,...
356,TS356,透明工厂,ts
357,TS357,人造肉,ts
358,TS358,垃圾分类,ts
359,TS359,科创板,ts


In [15]:
df = pro.concept_detail(ts_code='000001.SZ')

In [16]:
df

,id,concept_name,ts_code,name
0,TS11,银行,000001.SZ,平安银行
1,TS25,MSCI,000001.SZ,平安银行
2,TS123,供应链金融,000001.SZ,平安银行
